# Baseline model evaluation

In [ ]:
import os
import respiratory_extraction.utils as utils

data_path = os.path.join(os.getcwd(), '..', 'data', 'subjects')
dataset = utils.Dataset(data_path)

subjects = dataset.get_subjects()
subjects

In [ ]:
# The scenarios in which the subjects were recorded
# scenarios = dataset.get_scenarios()
scenarios = [
    '101_natural_lighting',
    # '102_artificial_lighting',
]
scenarios

In [ ]:
from datetime import datetime

hyperparameters = {
    # The minimum frequency to consider in the FFT. Default is 0.1 Hz (6 breaths per minute)
    'min_freq': 0.1,
    # The maximum frequency to consider in the FFT. Default is 0.6 Hz (36 breaths per minute)
    'max_freq': 0.6,
}

evaluation_metadata = {
    'model': 'baseline',
    'start_time': datetime.now(),
    'hyperparameters': hyperparameters,
    'subjects': subjects,
    'scenarios': scenarios,
}

## Display the first frame of each scenario for the first subject

In [ ]:
import respiratory_extraction.models.baseline as baseline

evaluation_entries = []
faulty_scenarios = []

for subject in subjects:
    for idx, scenario in enumerate(scenarios):
        print(f'Processing {subject} - {scenario}')

        frames, params = dataset.read_video_gray(subject, scenario)

        faces = baseline.face_detection(frames[0])
        if len(faces) != 1:
            print(f'No face detected in {subject} - {scenario}')
            faulty_scenarios.append((subject, scenario))
            continue

        # First face position and size
        face1 = faces[0]

        # Calculate the region of interest (ROI) based on the face
        roi = baseline.calculate_roi_from_face(face1[0], face1[1], face1[2], face1[3])

        # Calculate the average pixel intensity in the chest region
        average_pixels = baseline.average_pixel_intensity(frames, roi)

        # Calculate the fast fourier transform of the average pixel intensity
        pixels_fft, freq = baseline.calculate_fft(
            average_pixels,
            params.fps,
            min_freq=hyperparameters['min_freq'],
            max_freq=hyperparameters['max_freq'],
        )

        # Calculate the respiratory rate
        max_freq, breathing_rate = baseline.calculate_respiratory_rate(pixels_fft, freq)

        # Read the ground truth data from unisens directory
        gt_signal, gt_sample_rate = dataset.read_unisens_entry(subject, scenario, '3_Thorax')
        # Calculate the fast fourier transform of the ground truth signal
        gt_fft, gt_freq = baseline.calculate_fft(gt_signal, gt_sample_rate)
        gt_max_freq, gt_breathing_rate = baseline.calculate_respiratory_rate(gt_fft, gt_freq)

        # Append the evaluation entry
        evaluation_entries.append({
            'subject': subject,
            'scenario': scenario,
            'estimated_respiration_rate': breathing_rate,
            'estimated_freq': max_freq,
            'gt_respiration_rate': gt_breathing_rate,
            'gt_freq': gt_max_freq,
        })

        # Garbage collect the frames
        del frames

In [ ]:
faulty_scenarios

In [ ]:
evaluation_metadata['end_time'] = datetime.now()

In [ ]:
import pandas as pd

evaluation_df = pd.DataFrame(evaluation_entries)
evaluation_df.head()

In [ ]:
evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'baseline')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

# Save the evaluation dataframe
csv_path = os.path.join(evaluation_dir, 'baseline_evaluation.csv')
evaluation_df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(evaluation_dir, 'baseline_hyperparameters.json')
utils.write_json(json_path, evaluation_metadata)